# Predicting UFC Winners

## Overview

## Business Problem

## Data Understanding

In [193]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [245]:
# Importing everything needed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, precision_score, ConfusionMatrixDisplay, 
recall_score, confusion_matrix, precision_score, make_scorer)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

In [195]:
# Read in the data and display a preview
df = pd.read_csv('../data/ufc-master.csv')
df.head()

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,title_bout,weight_class,gender,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_landed,B_avg_TD_pct,B_longest_win_streak,B_losses,B_total_rounds_fought,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_current_lose_streak,R_current_win_streak,R_draw,R_avg_SIG_STR_landed,R_avg_SIG_STR_pct,R_avg_SUB_ATT,R_avg_TD_landed,R_avg_TD_pct,R_longest_win_streak,R_losses,R_total_rounds_fought,R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,B_age,lose_streak_dif,win_streak_dif,longest_win_streak_dif,win_dif,loss_dif,total_round_dif,total_title_bout_dif,ko_dif,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,B_match_weightclass_rank,R_match_weightclass_rank,R_Women's Flyweight_rank,R_Women's Featherweight_rank,R_Women's Strawweight_rank,R_Women's Bantamweight_rank,R_Heavyweight_rank,R_Light Heavyweight_rank,R_Middleweight_rank,R_Welterweight_rank,R_Lightweight_rank,R_Featherweight_rank,R_Bantamweight_rank,R_Flyweight_rank,R_Pound-for-Pound_rank,B_Women's Flyweight_rank,B_Women's Featherweight_rank,B_Women's Strawweight_rank,B_Women's Bantamweight_rank,B_Heavyweight_rank,B_Light Heavyweight_rank,B_Middleweight_rank,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank,finish,finish_details,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds
0,Thiago Santos,Johnny Walker,-150.0,130,66.666667,130.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,False,Light Heavyweight,MALE,5,0,1,0,3.42,0.59,0.7,0.24,1.00,4,2,11,0,0,0,1,4,0,0,5,Orthodox,198.12,208.28,205,3,0,0,3.95,0.48,0.1,0.61,0.36,4,8,43,1,0,0,2,11,0,0,13,Orthodox,187.96,193.04,205,37,29,-3,1,0,-8,-6,-32,-1,-7,0,10.16,15.24,-8,-0.53,0.6,-0.37,1,1,10.0,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red,U-DEC,NaN,5.0,5:00,1500.0,800.0,900.0,2000.0,1600.0,-110.0,175.0
1,Alex Oliveira,Niko Price,170.0,-200,170.000000,50.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,False,Welterweight,MALE,3,2,0,0,5.16,0.42,0.8,0.79,0.22,2,5,24,0,0,0,0,4,2,0,6,Orthodox,182.88,193.04,170,2,0,0,2.97,0.51,0.5,2.27,0.37,4,8,44,0,0,1,3,4,3,0,11,Orthodox,180.34,193.04,170,33,32,0,0,-2,-5,-3,-20,0,0,-1,2.54,0.00,-1,2.19,0.3,-1.48,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,U-DEC,NaN,3.0,5:00,900.0,450.0,350.0,700.0,1100.0,550.0,120.0
2,Misha Cirkunov,Krzysztof Jotko,110.0,-130,110.000000,76.923077,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,False,Middleweight,MALE,3,1,0,0,2.92,0.41,0.1,1.15,0.34,5,5,38,0,0,2,6,1,0,0,9,Southpaw,185.42,195.58,185,1,0,0,3.77,0.49,1.7,4.48,0.44,4,4,13,0,0,0,0,1,5,0,6,Orthodox,190.50,195.58,205,34,32,0,0,1,3,1,25,0,0,-5,-5.08,0.00,-2,-0.85,-1.6,-3.33,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,S-DEC,NaN,3.0,5:00,900.0,550.0,275.0,275.0,1400.0,600.0,185.0
3,Alexander Hernandez,Mike Breeden,-675.0,475,14.814815,475.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,False,Lightweight,MALE,3,1,0,0,4.04,0.34,0.0,0.00,0.00,0,1,3,0,0,0,0,0,0,0,0,Orthodox,177.80,177.80,155,1,0,0,3.79,0.36,0.0,1.57,0.35,2,3,15,0,0,0,2,2,0,0,4,Orthodox,175.26,182.88,155,29,32,0,0,-2,-4,-2,-12,0,-2,0,2.54,-5.08,3,0.25,0.0,-1.57,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [196]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4896 entries, 0 to 4895
Data columns (total 119 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   R_fighter                     object 
 1   B_fighter                     object 
 2   R_odds                        float64
 3   B_odds                        int64  
 4   R_ev                          float64
 5   B_ev                          float64
 6   date                          object 
 7   location                      object 
 8   country                       object 
 9   Winner                        object 
 10  title_bout                    bool   
 11  weight_class                  object 
 12  gender                        object 
 13  no_of_rounds                  int64  
 14  B_current_lose_streak         int64  
 15  B_current_win_streak          int64  
 16  B_draw                        int64  
 17  B_avg_SIG_STR_landed          float64
 18  B_avg_SIG_STR_pct          

## Data Preparation

I want to change date column to a datetime object but they are in different formats

In [197]:
df['date']

0       2021-10-02
1       2021-10-02
2       2021-10-02
3       2021-10-02
4       2021-10-02
5       2021-10-02
6       2021-10-02
7       2021-10-02
8       2021-10-02
9       2021-10-02
10      2021-10-02
11      2021-09-25
12      2021-09-25
13      2021-09-25
14      2021-09-25
15      2021-09-25
16      2021-09-25
17      2021-09-25
18      2021-09-25
19      2021-09-25
20      2021-09-25
21      2021-09-25
22      2021-09-25
23      2021-09-25
24      2021-09-18
25      2021-09-18
26      2021-09-18
27      2021-09-18
28      2021-09-18
29      2021-09-18
30      2021-09-18
31      2021-09-18
32      2021-09-18
33      2021-09-18
34      2021-09-18
35      2021-09-18
36      2021-09-18
37      2021-09-04
38      2021-09-04
39      2021-09-04
40      2021-09-04
41      2021-09-04
42      2021-09-04
43      2021-09-04
44      2021-09-04
45      2021-09-04
46      2021-08-28
47      2021-08-28
48      2021-08-28
49      2021-08-28
50      2021-08-28
51      2021-08-28
52      2021

In [198]:
# Changing all / to - in date column
df['date'] = df['date'].str.replace('/', '-')

The first 58 dates are in a different format. They have year month day instead of month day year.

In [199]:
df['date'][:59]

0     2021-10-02
1     2021-10-02
2     2021-10-02
3     2021-10-02
4     2021-10-02
5     2021-10-02
6     2021-10-02
7     2021-10-02
8     2021-10-02
9     2021-10-02
10    2021-10-02
11    2021-09-25
12    2021-09-25
13    2021-09-25
14    2021-09-25
15    2021-09-25
16    2021-09-25
17    2021-09-25
18    2021-09-25
19    2021-09-25
20    2021-09-25
21    2021-09-25
22    2021-09-25
23    2021-09-25
24    2021-09-18
25    2021-09-18
26    2021-09-18
27    2021-09-18
28    2021-09-18
29    2021-09-18
30    2021-09-18
31    2021-09-18
32    2021-09-18
33    2021-09-18
34    2021-09-18
35    2021-09-18
36    2021-09-18
37    2021-09-04
38    2021-09-04
39    2021-09-04
40    2021-09-04
41    2021-09-04
42    2021-09-04
43    2021-09-04
44    2021-09-04
45    2021-09-04
46    2021-08-28
47    2021-08-28
48    2021-08-28
49    2021-08-28
50    2021-08-28
51    2021-08-28
52    2021-08-28
53    2021-08-28
54    2021-08-28
55    2021-08-28
56    2021-08-28
57    2021-08-28
58     8-21-20

In [200]:
# Only want to change the first 58 values
for x in range(58):
    # Accessing each value in date column
    string = df.loc[x, 'date']
    # Slicing to get day, month, and year separated
    day = string[8:]
    month = string[5:7]
    year = string[:4]
    # Moving components around to get month-day-year format
    new_format = month + '-' + day + '-' + year
    # Update the DataFrame
    df.loc[x, 'date'] = new_format

In [201]:
df['date'][:59]

0     10-02-2021
1     10-02-2021
2     10-02-2021
3     10-02-2021
4     10-02-2021
5     10-02-2021
6     10-02-2021
7     10-02-2021
8     10-02-2021
9     10-02-2021
10    10-02-2021
11    09-25-2021
12    09-25-2021
13    09-25-2021
14    09-25-2021
15    09-25-2021
16    09-25-2021
17    09-25-2021
18    09-25-2021
19    09-25-2021
20    09-25-2021
21    09-25-2021
22    09-25-2021
23    09-25-2021
24    09-18-2021
25    09-18-2021
26    09-18-2021
27    09-18-2021
28    09-18-2021
29    09-18-2021
30    09-18-2021
31    09-18-2021
32    09-18-2021
33    09-18-2021
34    09-18-2021
35    09-18-2021
36    09-18-2021
37    09-04-2021
38    09-04-2021
39    09-04-2021
40    09-04-2021
41    09-04-2021
42    09-04-2021
43    09-04-2021
44    09-04-2021
45    09-04-2021
46    08-28-2021
47    08-28-2021
48    08-28-2021
49    08-28-2021
50    08-28-2021
51    08-28-2021
52    08-28-2021
53    08-28-2021
54    08-28-2021
55    08-28-2021
56    08-28-2021
57    08-28-2021
58     8-21-20

In [202]:
# Converting column to datetime from object
df['date'] = pd.to_datetime(df['date'], format='%m-%d-%Y')

In [203]:
df['date']

0      2021-10-02
1      2021-10-02
2      2021-10-02
3      2021-10-02
4      2021-10-02
5      2021-10-02
6      2021-10-02
7      2021-10-02
8      2021-10-02
9      2021-10-02
10     2021-10-02
11     2021-09-25
12     2021-09-25
13     2021-09-25
14     2021-09-25
15     2021-09-25
16     2021-09-25
17     2021-09-25
18     2021-09-25
19     2021-09-25
20     2021-09-25
21     2021-09-25
22     2021-09-25
23     2021-09-25
24     2021-09-18
25     2021-09-18
26     2021-09-18
27     2021-09-18
28     2021-09-18
29     2021-09-18
30     2021-09-18
31     2021-09-18
32     2021-09-18
33     2021-09-18
34     2021-09-18
35     2021-09-18
36     2021-09-18
37     2021-09-04
38     2021-09-04
39     2021-09-04
40     2021-09-04
41     2021-09-04
42     2021-09-04
43     2021-09-04
44     2021-09-04
45     2021-09-04
46     2021-08-28
47     2021-08-28
48     2021-08-28
49     2021-08-28
50     2021-08-28
51     2021-08-28
52     2021-08-28
53     2021-08-28
54     2021-08-28
55     202

In [204]:
df.describe()

,R_odds,B_odds,R_ev,B_ev,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_landed,B_avg_TD_pct,B_longest_win_streak,B_losses,B_total_rounds_fought,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_current_lose_streak,R_current_win_streak,R_draw,R_avg_SIG_STR_landed,R_avg_SIG_STR_pct,R_avg_SUB_ATT,R_avg_TD_landed,R_avg_TD_pct,R_longest_win_streak,R_losses,R_total_rounds_fought,R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,B_age,lose_streak_dif,win_streak_dif,longest_win_streak_dif,win_dif,loss_dif,total_round_dif,total_title_bout_dif,ko_dif,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,B_match_weightclass_rank,R_match_weightclass_rank,R_Women's Flyweight_rank,R_Women's Featherweight_rank,R_Women's Strawweight_rank,R_Women's Bantamweight_rank,R_Heavyweight_rank,R_Light Heavyweight_rank,R_Middleweight_rank,R_Welterweight_rank,R_Lightweight_rank,R_Featherweight_rank,R_Bantamweight_rank,R_Flyweight_rank,R_Pound-for-Pound_rank,B_Women's Flyweight_rank,B_Women's Featherweight_rank,B_Women's Strawweight_rank,B_Women's Bantamweight_rank,B_Heavyweight_rank,B_Light Heavyweight_rank,B_Middleweight_rank,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,finish_round,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds
count,4895.000000,4896.000000,4895.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,3966.000000,4131.000000,4064.000000,4063.000000,4054.000000,4896.00000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4441.000000,4539.000000,4539.000000,4539.000000,4529.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.0,877.000000,1328.000000,59.000000,7.0,104.000000,118.000000,142.000000,141.000000,134.000000,140.000000,139.000000,133.000000,137.000000,142.000000,166.000000,44.000000,0.0,61.000000,78.000000,110.000000,93.000000,102.000000,89.000000,87.000000,90.000000,85.000000,95.000000,35.000000,4274.000000,4274.000000,4093.000000,4077.000000,3847.000000,3835.000000,3847.000000,3834.000000
mean,-117.640449,66.030637,94.827397,167.083323,3.181985,0.477941,0.875408,0.010621,26.308553,0.444741,0.504614,1.251173,0.312506,1.76491,1.726511,10.742443,0.232639,0.014502,0.260417,0.975490,0.960989,0.588848,0.029616,2.880923,178.084669,182.435958,164.803105,0.613154,1.051062,0.013480,27.396408,0.454709,0.517659,1.337388,0.329411,2.530842,2.393587,15.950980,0.552083,0.025735,0.365809,1.468750,1.449551,0.866422,0.047386,4.293709,178.030948,182.619835,165.253472,30.056985,29.518382,0.105392,-0.175449,-0.765931,-1.412786,0.370507,-5.208538,-0.319444,-0.506332,-0.277574,0.018591,-0.251172,0.329044,-3.539160,-0.061050,-0.201570,0.138480,1.0,8.262258,6.834337,6.677966,0.0,6.826923,7.050847,6.774648,6.673759,7.649254,6.807143,7.093525,6.827068,6.985401,6.457746,6.789157,7.795455,NaN,7.803279,8.192308,8.490909,8.268817,8.745098,8.179775,8.126437,8.022222,8.388235,8.473684,9.485714,2.408049,652.313758,294.064745,416.544027,843.010138,1064.543155,514.231869,647.257173
std,268.881452,247.803928,82.843409,136.944

In [205]:
df.isna().sum()

R_fighter                          0
B_fighter                          0
R_odds                             1
B_odds                             0
R_ev                               1
B_ev                               0
date                               0
location                           0
country                            0
Winner                             0
title_bout                         0
weight_class                       0
gender                             0
no_of_rounds                       0
B_current_lose_streak              0
B_current_win_streak               0
B_draw                             0
B_avg_SIG_STR_landed             930
B_avg_SIG_STR_pct                765
B_avg_SUB_ATT                    832
B_avg_TD_landed                  833
B_avg_TD_pct                     842
B_longest_win_streak               0
B_losses                           0
B_total_rounds_fought              0
B_total_title_bouts                0
B_win_by_Decision_Majority         0
B

In [206]:
nulls = df.isna().sum()
nulls.sort_values(ascending=False)[:30]

B_Women's Featherweight_rank    4896
R_Women's Featherweight_rank    4889
B_Pound-for-Pound_rank          4861
B_Women's Flyweight_rank        4852
R_Women's Flyweight_rank        4837
B_Women's Strawweight_rank      4835
B_Women's Bantamweight_rank     4818
B_Bantamweight_rank             4811
B_Lightweight_rank              4809
B_Welterweight_rank             4807
B_Featherweight_rank            4806
B_Light Heavyweight_rank        4803
B_Flyweight_rank                4801
B_Middleweight_rank             4794
R_Women's Strawweight_rank      4792
B_Heavyweight_rank              4786
R_Women's Bantamweight_rank     4778
R_Featherweight_rank            4763
R_Middleweight_rank             4762
R_Bantamweight_rank             4759
R_Lightweight_rank              4757
R_Welterweight_rank             4756
R_Light Heavyweight_rank        4755
R_Heavyweight_rank              4754
R_Flyweight_rank                4754
R_Pound-for-Pound_rank          4730
B_match_weightclass_rank        4019
R

In [207]:
columns_to_drop = list(nulls.sort_values(ascending=False)[:29].index)
columns_to_drop

["B_Women's Featherweight_rank",
 "R_Women's Featherweight_rank",
 'B_Pound-for-Pound_rank',
 "B_Women's Flyweight_rank",
 "R_Women's Flyweight_rank",
 "B_Women's Strawweight_rank",
 "B_Women's Bantamweight_rank",
 'B_Bantamweight_rank',
 'B_Lightweight_rank',
 'B_Welterweight_rank',
 'B_Featherweight_rank',
 'B_Light Heavyweight_rank',
 'B_Flyweight_rank',
 'B_Middleweight_rank',
 "R_Women's Strawweight_rank",
 'B_Heavyweight_rank',
 "R_Women's Bantamweight_rank",
 'R_Featherweight_rank',
 'R_Middleweight_rank',
 'R_Bantamweight_rank',
 'R_Lightweight_rank',
 'R_Welterweight_rank',
 'R_Light Heavyweight_rank',
 'R_Heavyweight_rank',
 'R_Flyweight_rank',
 'R_Pound-for-Pound_rank',
 'B_match_weightclass_rank',
 'R_match_weightclass_rank',
 'finish_details']

In [208]:
df.drop(columns_to_drop, axis=1, inplace=True)

In [209]:
nulls2 = df.isna().sum()
nulls2.sort_values(ascending=False)

b_ko_odds                       1062
b_sub_odds                      1061
r_ko_odds                       1049
r_sub_odds                      1049
B_avg_SIG_STR_landed             930
B_avg_TD_pct                     842
B_avg_TD_landed                  833
B_avg_SUB_ATT                    832
b_dec_odds                       819
r_dec_odds                       803
B_avg_SIG_STR_pct                765
total_fight_time_secs            622
finish_round_time                622
finish_round                     622
R_avg_SIG_STR_landed             455
R_avg_TD_pct                     367
R_avg_TD_landed                  357
R_avg_SIG_STR_pct                357
R_avg_SUB_ATT                    357
finish                           238
B_Stance                           2
R_ev                               1
R_odds                             1
B_win_by_Submission                0
B_win_by_KO/TKO                    0
B_win_by_TKO_Doctor_Stoppage       0
empty_arena                        0
B

In [210]:
clean_df = df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2615 entries, 0 to 4286
Data columns (total 90 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   R_fighter                     2615 non-null   object        
 1   B_fighter                     2615 non-null   object        
 2   R_odds                        2615 non-null   float64       
 3   B_odds                        2615 non-null   int64         
 4   R_ev                          2615 non-null   float64       
 5   B_ev                          2615 non-null   float64       
 6   date                          2615 non-null   datetime64[ns]
 7   location                      2615 non-null   object        
 8   country                       2615 non-null   object        
 9   Winner                        2615 non-null   object        
 10  title_bout                    2615 non-null   bool          
 11  weight_class                  

In [211]:
# Creating a column that has the name of the underdog by looking at the odds columns
clean_df['underdog'] = clean_df.apply(
    lambda row: row['R_fighter'] if row['R_odds'] > row['B_odds'] else row['B_fighter'], axis=1)
# Creating a column that has the name of the favored fighter by looking at the odds columns
clean_df['favored'] = clean_df.apply(
    lambda row: row['R_fighter'] if row['R_odds'] < row['B_odds'] else row['B_fighter'], axis=1)

<ipython-input-211-f1a5e82c79dc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['underdog'] = clean_df.apply(
<ipython-input-211-f1a5e82c79dc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['favored'] = clean_df.apply(


In [212]:
clean_df['underdog']

0                   Johnny Walker
1                   Alex Oliveira
2                  Misha Cirkunov
3                    Mike Breeden
4                    Jared Gordon
5             Antonina Shevchenko
6                   Bethe Correia
7                 Jamie Mullarkey
8                Gaetano Pirrello
9                    Shanna Young
10                 Johnny Eduardo
11                   Brian Ortega
12                  Lauren Murphy
13                      Nick Diaz
14          Jairzinho Rozenstruik
15               Cynthia Calvillo
16                  Marlon Moraes
17               Nasrat Haqparast
18            Shamil Abdurakhimov
19              Roxanne Modafferi
20                   Jalin Turner
21                  Cody Brundage
23                Jonathan Pearce
24                     Ryan Spann
25                    Devin Clark
26                     Mandy Bohm
27                Christos Giagos
28                    Nate Maness
29                 Antonio Arroyo
30            

In [213]:
clean_df['Winner']

0        Red
1       Blue
2       Blue
3        Red
4       Blue
5       Blue
6       Blue
7       Blue
8        Red
9        Red
10       Red
11       Red
12       Red
13      Blue
14       Red
15       Red
16      Blue
17       Red
18      Blue
19      Blue
20      Blue
21      Blue
23       Red
24       Red
25       Red
26       Red
27       Red
28       Red
29       Red
30      Blue
31      Blue
33       Red
34       Red
35      Blue
36      Blue
37       Red
38       Red
39       Red
40      Blue
41       Red
42       Red
43       Red
44       Red
45      Blue
46      Blue
47       Red
48       Red
49      Blue
50       Red
51      Blue
52      Blue
53      Blue
54       Red
55       Red
56      Blue
57       Red
58       Red
59      Blue
60       Red
61      Blue
62       Red
63       Red
64       Red
65       Red
66      Blue
67       Red
68      Blue
69      Blue
70      Blue
71       Red
72      Blue
73       Red
74       Red
75      Blue
76       Red
77       Red
78      Blue

In [214]:
# Changing Red winners to either underdog or favored
clean_df.loc[(clean_df['Winner'] == 'Red') & (clean_df['R_fighter'] == clean_df['underdog']), 'Winner'] = 'underdog'
clean_df.loc[(clean_df['Winner'] == 'Red') & (clean_df['R_fighter'] != clean_df['underdog']), 'Winner'] = 'favored'
# Changing Blue winners to either underdog or favored
clean_df.loc[(clean_df['Winner'] == 'Blue') & (clean_df['B_fighter'] == clean_df['underdog']), 'Winner'] = 'underdog'
clean_df.loc[(clean_df['Winner'] == 'Blue') & (clean_df['B_fighter'] != clean_df['underdog']), 'Winner'] = 'favored'

C:\Users\42ben\anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [215]:
clean_df['Winner'].value_counts(normalize=True)

favored     0.640918
underdog    0.359082
Name: Winner, dtype: float64

Wanted to compare this to the whole dataset before dropping nulls just to make sure it was close to the same amount of winners in each category.

In [216]:
df['R_odds'].dropna(inplace=True)
df['B_odds'].dropna(inplace=True)
df['R_fighter'].dropna(inplace=True)
df['B_fighter'].dropna(inplace=True)

# Creating a column that has the name of the underdog by looking at the odds columns
df['underdog'] = df.apply(
    lambda row: row['R_fighter'] if row['R_odds'] > row['B_odds'] else row['B_fighter'], axis=1)
# Creating a column that has the name of the favored fighter by looking at the odds columns
df['favored'] = df.apply(
    lambda row: row['R_fighter'] if row['R_odds'] < row['B_odds'] else row['B_fighter'], axis=1)

# Changing Red winners to either underdog or favored
df.loc[(df['Winner'] == 'Red') & (df['R_fighter'] == df['underdog']), 'Winner'] = 'underdog'
df.loc[(df['Winner'] == 'Red') & (df['R_fighter'] != df['underdog']), 'Winner'] = 'favored'
# Changing Blue winners to either underdog or favored
df.loc[(df['Winner'] == 'Blue') & (df['B_fighter'] == df['underdog']), 'Winner'] = 'underdog'
df.loc[(df['Winner'] == 'Blue') & (df['B_fighter'] != df['underdog']), 'Winner'] = 'favored'

df['Winner'].value_counts(normalize=True)

favored     0.652982
underdog    0.347018
Name: Winner, dtype: float64

Running a .corr() in order to see which features are the most correlated with the target column.

In [217]:
# Creating a copy of clean_df
correlation = clean_df.copy()
# Changing 'Winner' to numeric data
correlation['Winner'] = correlation['Winner'].map({'favored': 0, 'underdog': 1})
# Changing Stance columns for each fighter to numeric data
correlation['R_Stance'] = correlation['R_Stance'].map({'Orthodox': 0, 'Southpaw': 1, 'Switch': 2})
correlation['B_Stance'] = correlation['B_Stance'].map({'Orthodox': 0, 'Southpaw': 1, 'Switch': 2})
# Running .corr on table
correlation.corr()

,R_odds,B_odds,R_ev,B_ev,Winner,title_bout,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_landed,B_avg_TD_pct,B_longest_win_streak,B_losses,B_total_rounds_fought,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_current_lose_streak,R_current_win_streak,R_draw,R_avg_SIG_STR_landed,R_avg_SIG_STR_pct,R_avg_SUB_ATT,R_avg_TD_landed,R_avg_TD_pct,R_longest_win_streak,R_losses,R_total_rounds_fought,R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,B_age,lose_streak_dif,win_streak_dif,longest_win_streak_dif,win_dif,loss_dif,total_round_dif,total_title_bout_dif,ko_dif,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,finish_round,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds
R_odds,1.000000,-0.969696,0.820485,-0.936470,0.137393,-0.193818,-0.157417,-0.027434,0.024014,0.010812,0.037547,0.107452,0.001564,0.061750,0.070163,0.019991,-0.116263,-0.075890,-0.041722,0.006451,-0.091243,-0.031355,-0.006493,-0.017311,-0.025026,-0.038508,0.022642,0.075444,0.091361,0.040392,0.155098,-0.230500,0.019866,-0.078147,-0.149632,0.013216,-0.142272,-0.090543,-0.116238,0.268088,0.112299,-0.145455,0.010071,0.117675,0.014449,0.024067,0.002454,0.017848,0.043791,-0.006977,0.026245,-0.006578,0.023716,0.257783,-0.231080,0.121741,0.242356,0.137538,-0.078277,0.229937,-0.166741,0.105383,-0.032795,-0.014400,0.068485,0.116145,0.303529,0.113950,-0.008549,0.150206,-0.000842,NaN,-0.036908,-0.019921,0.356514,-0.675676,0.324984,-0.392236,0.508780,-0.601083
B_odds,-0.969696,1.000000,-0.878323,0.892632,-0.120096,0.174785,0.152741,0.036489,-0.035938,-0.006048,-0.038494,-0.106750,0.001309,-0.074180,-0.076900,-0.016015,0.130088,0.083698,0.050921,0.001038,0.083669,0.031780,0.023880,0.022655,0.029532,0.047064,-0.025750,-0.074445,-0.089382,-0.037087,-0.159249,0.213813,-0.016613,0.086606,0.143816,-0.006625,0.134616,0.087777,0.108731,-0.270936,-0.118751,0.128089,-0.010754,-0.117158,-0.015878,-0.022690,-0.008597,-0.015095,-0.046405,0.012491,-0.022717,0.008189,-0.019256,-0.268070,0.242890,-0.131193,-0.234786,-0.126292,0.088477,-0.238107,0.179087,-0.084116,0.046025,0.023298,-0.072124,-0.116223,-0.316121,-0.123093,0.005881,-0.153856,-0.009896,NaN,0.035872,0.020138,-0.395906,0.640760,-0.338724,0.382485,-0.531917,0.586146
R_ev,0.820485,-0.878323,1.000000,-0.639952,0.029789,-0.116346,-0.117465,-0.043147,0.046130,-0.008782,0.067026,0.094396,-0.005516,0.087359,0.071865,-0.002224,-0.146867,-0.102307,-0.058296,-0.003373,-0.079102,-0.033459,-0.054250,-0.029453,-0.021915,-0.068539,0.019210,0.055255,0.072532,0.015748,0.141065,-0.159589,0.007877,-0.042776,-0.120090,-0.015059,-0.099918,-0.082339,-0.087252,0.244971,0.113477,-0.063748,0.008052,0.112737,0.020362,0.013215,0.008109,0.029297,0.041094,-0.021147,-0.001627,-0.023116,0.001174,0.259020,-0.232673,0.121048,0.189340,0.088084,-0.102078,0.229523,-0.189044,0.021677,-0.062985,-0.027753,0.079965,0.114561,0.299660,0.109959,0.006895,0.138255,-0.028255,NaN,-0.053456,-0.048283,0.463268,-0.466056,0.345334,-0.303920,0.532878,-0.476202
B_ev,-0.936470,0.892632,-0.639952,1.000000,-0.177425,0.214087,0.171286,0.010087,-0.008161,-0.024703,-0.009823,-0.099328,0.001440,-0.045657,-0.058359,-0.021500,0.088432,0.054355,0.033748,-0.009951,0.079611,0.030005,-0.006678,0.001766,0.034746,0.022818,-0.029635,-0.075963,-0.089074,-0.046263,-0.142677,0.241733,-0.032104,0.104730,0.142564,-0.019921,0.148609,0.086933,0.123562,-0.253396,-0.103146,0.178077,-0.022669,-0.107901,-0.008555,-0.026644,-0.000102,-0.012800,-0.040112,-0.001710,-0.036277,-0.004637,-0.031679,-0.227254,0.204388,

In [218]:
clean_df['no_of_rounds'].value_counts()

3    2294
5     320
4       1
Name: no_of_rounds, dtype: int64

Dropped this row because there is no such thing as a 4 round fight. They are either 3 or 5 rounds long.

In [219]:
# Dropping this row
clean_df.drop(clean_df.loc[clean_df['no_of_rounds'] == 4].index, inplace=True)

C:\Users\42ben\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Only using numeric features for now.

In [220]:
# Setting all wanted features equal to X
X = clean_df[['B_odds', 'R_odds', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous',
                    'win_streak_dif', 'loss_dif', 'reach_dif', 'age_dif', 'r_dec_odds', 'b_dec_odds',
                  'b_sub_odds', 'r_ko_odds', 'b_ko_odds']]
# Setting target equal to y
y = clean_df['Winner']
# Performing the train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=333)

## Modeling

## Evaluation

A dummy model would pick the underdog every time and be correct about 36% of the time. This also displays an expected yet slight class imbalance, nothing dramatic.

In [221]:
y_train.value_counts(normalize=True)

favored     0.637245
underdog    0.362755
Name: Winner, dtype: float64

### First Model (Decision Tree)
For my first model created a decision tree with all default hyperparameters to see if it would over fit to the training data and score a perfect 1.0 

In [222]:
# Creating steps for a Pipeline
tree_steps = [('scaler', StandardScaler()),
             ('tree', DecisionTreeClassifier(random_state=333))]
# Creating Pipeline with steps
tree_pipe = Pipeline(tree_steps)
# Fitting the training data to the Pipeline
tree_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('tree', DecisionTreeClassifier(random_state=333))])

It did get a perfect score on the training data which gives me confidence I'm working with data that is capable of creating a sufficient model.

In [223]:
# Defining the predictions from the Pipeline using the training data
y_pred = tree_pipe.predict(X_train)
# Evaluating the accuracy score on the training data
accuracy_score(y_train, y_pred), precision_score(y_train, y_pred, pos_label='underdog')

(1.0, 1.0)

The cross validation accuracy score is pretty good for a first model with all default hyperparameters. It is about 20% higher than the dummy model. 

In [224]:
# Calculating mean of the 5-fold cross_val_score on accuracy
cross_val_score(tree_pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.5688775510204082

I made a custom scorer for precision to use when cross validating because this is the metric I'm mostly concerned with.

In [225]:
# Making a custom scorer for precision
precision_scorer = make_scorer(precision_score, pos_label='underdog')

This precision score is also not too bad for the first model at 41%

In [226]:
# Calculating mean of the 5-fold cross_val_score on precision
cross_val_score(tree_pipe, X_train, y_train, cv=5, scoring=precision_scorer).mean()

0.4092745140796266

### 1st Model (Random Forest)

In [227]:
# Creating steps for a Pipeline
forest_steps = [('scaler', StandardScaler()),
             ('forest', RandomForestClassifier(random_state=333))]
# Creating Pipeline with steps
forest_pipe = Pipeline(forest_steps)
# Fitting the training data to the Pipeline
forest_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('forest', RandomForestClassifier(random_state=333))])

In [228]:
# Defining the predictions from the Pipeline using the training data
y_pred = forest_pipe.predict(X_train)
# Evaluating the accuracy and precision score on the training data
accuracy_score(y_train, y_pred), precision_score(y_train, y_pred, pos_label='underdog')

(1.0, 1.0)

In [229]:
# Calculating mean of the 5-fold cross_val_score on precision
cross_val_score(tree_pipe, X_train, y_train, cv=5, scoring=precision_scorer).mean()

0.4092745140796266

### Random Forest 2nd iteration (Using GridSearch)

In [230]:
# Creating parameters for GridSearch
forest_params = {'forest__n_estimators': [50, 100, 150],
                'forest__max_depth': [5, 50, 100],
                'forest__class_weight': ['balanced', None],
                'forest__criterion': ['gini', 'entropy', 'log_loss']}
# GridSearch with the random forest pipeline, parameters above, 5 fold cross validation, and precision score
forest_grid = GridSearchCV(estimator=forest_pipe, param_grid=forest_params, cv=5, scoring=precision_scorer, n_jobs=3)
# Fitting the GridSearch
forest_grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('forest',
                                        RandomForestClassifier(random_state=333))]),
             n_jobs=3,
             param_grid={'forest__class_weight': ['balanced', None],
                         'forest__criterion': ['gini', 'entropy', 'log_loss'],
                         'forest__max_depth': [5, 50, 100],
                         'forest__n_estimators': [50, 100, 150]},
             return_train_score=True,
             scoring=make_scorer(precision_score, pos_label=underdog))

In [237]:
# Displaying the best estimator from the GridSearch and the accuracy score
print(forest_grid.best_estimator_)
print(forest_grid.best_score_)

Pipeline(steps=[('scaler', StandardScaler()),
                ('forest',
                 RandomForestClassifier(criterion='entropy', max_depth=50,
                                        n_estimators=150, random_state=333))])
0.49200305906061487


In [238]:
forest_grid.best_estimator_.score(X_train, y_train)

1.0

### Random Forest 3rd iteration (Using GridSearch)

In [242]:
# Creating parameters for GridSearch
forest_params2 = {'forest__n_estimators': [140, 150, 160],
                 'forest__max_depth': [45, 50, 75],
                 'forest__class_weight': [{'favored': 1, 'underdog': 2}, None],
                 'forest__criterion': ['gini', 'entropy', 'log_loss']}
# GridSearch with the random forest pipeline, parameters above, 5 fold cross validation, and precision score
forest_grid2 = GridSearchCV(estimator=forest_pipe, param_grid=forest_params2, cv=5, scoring=precision_scorer, n_jobs=3)
# Fitting the GridSearch
forest_grid2.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('forest',
                                        RandomForestClassifier(random_state=333))]),
             n_jobs=3,
             param_grid={'forest__class_weight': [{'favored': 1, 'underdog': 2},
                                                  None],
                         'forest__criterion': ['gini', 'entropy', 'log_loss'],
                         'forest__max_depth': [45, 50, 75],
                         'forest__n_estimators': [140, 150, 160]},
             scoring=make_scorer(precision_score, pos_label=underdog))

In [243]:
# Displaying the best estimator from the GridSearch and the accuracy score
print(forest_grid2.best_estimator_)
print(forest_grid2.best_score_)

Pipeline(steps=[('scaler', StandardScaler()),
                ('forest',
                 RandomForestClassifier(criterion='entropy', max_depth=45,
                                        n_estimators=150, random_state=333))])
0.49200305906061487


In [244]:
forest_grid2.best_estimator_.score(X_train, y_train)

1.0

### Random Forest with PCA

The model's precision is score increased but remained extremely overfit. PCA is a tactic used for dimensionality reduction which helps reduce overfitting. 

In [254]:
# Creating steps for a Pipeline w/ PCA
pca_forest_steps = [('scaler', StandardScaler()),
                    ('pca', PCA()),
                    ('forest', RandomForestClassifier(random_state=333, criterion='entropy'))]
# Creating Pipeline with steps
pca_forest_pipe = Pipeline(pca_forest_steps)
# Fitting the training data to the Pipeline
pca_forest_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA()),
                ('forest',
                 RandomForestClassifier(criterion='entropy',
                                        random_state=333))])

In [249]:
# Creating parameters for GridSearch w/ PCA
forest_params3 = {'pca__n_components': [2, 0.8, 0.9],
                  'forest__n_estimators': [50, 100, 150],
                  'forest__max_depth': [5, 50, 100],
                  'forest__class_weight': ['balanced', None],
                  'forest__criterion': ['gini', 'entropy', 'log_loss']}
# GridSearch with the random forest pipeline, parameters above, 5 fold cross validation, and precision score
forest_grid3 = GridSearchCV(estimator=pca_forest_pipe, param_grid=forest_params3, cv=5, scoring=precision_scorer, n_jobs=3)
# Fitting the GridSearch
forest_grid3.fit(X_train, y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()),
                                       ('forest',
                                        RandomForestClassifier(random_state=333))]),
             n_jobs=3,
             param_grid={'forest__class_weight': ['balanced', None],
                         'forest__criterion': ['gini', 'entropy', 'log_loss'],
                         'forest__max_depth': [5, 50, 100],
                         'forest__n_estimators': [50, 100, 150],
                         'pca__n_components': [2, 0.8, 0.9]},
             scoring=make_scorer(precision_score, pos_label=underdog),
             verbose=3)

In [250]:
# Displaying the best estimator from the GridSearch and the accuracy score
print(forest_grid3.best_estimator_)
print(forest_grid3.best_score_)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=2)),
                ('forest',
                 RandomForestClassifier(criterion='entropy', max_depth=5,
                                        random_state=333))])
0.6


In [251]:
forest_grid3.best_estimator_.score(X_train, y_train)

0.6454081632653061

In [252]:
# Creating parameters for GridSearch w/ PCA
forest_params4 = {'pca__n_components': [2, 3, 4],
                  'forest__n_estimators': [90, 100, 110],
                  'forest__max_depth': [5, 10, 20]}
# GridSearch with the random forest pipeline, parameters above, 5 fold cross validation, and precision score
forest_grid4 = GridSearchCV(estimator=pca_forest_pipe, param_grid=forest_params3, cv=5, scoring=precision_scorer, n_jobs=3)
# Fitting the GridSearch
forest_grid4.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()),
                                       ('forest',
                                        RandomForestClassifier(random_state=333))]),
             n_jobs=3,
             param_grid={'forest__class_weight': ['balanced', None],
                         'forest__criterion': ['gini', 'entropy', 'log_loss'],
                         'forest__max_depth': [5, 50, 100],
                         'forest__n_estimators': [50, 100, 150],
                         'pca__n_components': [2, 0.8, 0.9]},
             scoring=make_scorer(precision_score, pos_label=underdog))

In [253]:
# Displaying the best estimator from the GridSearch and the accuracy score
print(forest_grid4.best_estimator_)
print(forest_grid4.best_score_)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=2)),
                ('forest',
                 RandomForestClassifier(criterion='entropy', max_depth=5,
                                        random_state=333))])
0.6


In [259]:
# Creating parameters for GridSearch w/ PCA
forest_params4 = {'pca__n_components': [2, 3],
                  'forest__n_estimators': [99, 100, 101],
                  'forest__max_depth': [4, 5, 6]}
# GridSearch with the random forest pipeline, parameters above, 5 fold cross validation, and precision score
forest_grid4 = GridSearchCV(estimator=pca_forest_pipe, param_grid=forest_params3, cv=5, scoring=precision_scorer, n_jobs=3)
# Fitting the GridSearch
forest_grid4.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()),
                                       ('forest',
                                        RandomForestClassifier(criterion='entropy',
                                                               random_state=333))]),
             n_jobs=3,
             param_grid={'forest__class_weight': ['balanced', None],
                         'forest__criterion': ['gini', 'entropy', 'log_loss'],
                         'forest__max_depth': [5, 50, 100],
                         'forest__n_estimators': [50, 100, 150],
                         'pca__n_components': [2, 0.8, 0.9]},
             scoring=make_scorer(precision_score, pos_label=underdog))

In [260]:
# Displaying the best estimator from the GridSearch and the accuracy score
print(forest_grid4.best_estimator_)
print(forest_grid4.best_score_)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=2)),
                ('forest',
                 RandomForestClassifier(criterion='entropy', max_depth=5,
                                        random_state=333))])
0.6


In [261]:
forest_grid4.best_estimator_.score(X_train, y_train)

0.6454081632653061

### Logistic Regression
I was unable to further improve the random forest model. Next I created a logistic regression model. First, without PCA then with PCA in a new pipeline.

In [272]:
# Creating steps for Pipeline
logreg_steps = [('scaler', StandardScaler()),
               ('logreg', LogisticRegression(random_state=333))]
# Feeding steps to Pipeline
logreg_pipe = Pipeline(logreg_steps)
# Fitting training data to the Pipeline
logreg_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('logreg', LogisticRegression(random_state=333))])

Something appeared to be wrong here. A precision score of 1.0 with a logistic regression that has all defualt hyperparameters doesn't seem right, even on the training data.

In [274]:
# Defining the predictions from the Pipeline using the training data
y_pred = logreg_pipe.predict(X_train)
# Evaluating the accuracy and precision score on the training data
accuracy_score(y_train, y_pred), precision_score(y_train, y_pred, pos_label='underdog')

(0.6377551020408163, 1.0)

Accuracy score was lower on the cross valuation but still not terrible.

In [277]:
# Calculating mean of the 5-fold cross_val_score on accuracy
cross_val_score(tree_pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.5688775510204082

This model never predicted an underdog to win, the precision score is abysmal.

In [275]:
# Calculating mean of the 5-fold cross_val_score on precision
cross_val_score(logreg_pipe, X_train, y_train, cv=5, scoring=precision_scorer).mean()

C:\Users\42ben\anaconda3\envs\learn-env\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\42ben\anaconda3\envs\learn-env\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\42ben\anaconda3\envs\learn-env\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.06666666666666667

### Logistic Regression (2nd Iteration)
Tried to use a GridSearch to improve the logistic regression. Biggest concern is the class weights given the model never predicted an underdog to win.

In [278]:
# Creating parameters for GridSearch
logreg_params = {'logreg__penalty': ['l2', 'l1', 'elasticnet'],
                'logreg__C': [0.1, 1, 10],
                'logreg__class_weight': [{'favored': 1, 'underdog': 10}, 'balanced', None],
                'logreg__solver': ['lbfgs', 'sag', 'saga', 'newton-cholesky'],
                'logreg__max_iter': [100, 250]}
# Creating GridSearch with params above, cv=5, precision score, and logistic regression pipeline
logreg_grid = GridSearchCV(estimator=logreg_pipe, param_grid=logreg_params, cv=5, scoring=precision_scorer, n_jobs=3)
# Fitting the GridSearch
logreg_grid.fit(X_train, y_train)

C:\Users\42ben\anaconda3\envs\learn-env\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
630 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\42ben\anaconda3\envs\learn-env\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\42ben\anaconda3\envs\learn-env\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\42ben\anaconda3\envs\learn-env\lib\site-packages\sklearn\pipeline.py", line 420, in fit
    self._final_

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('logreg',
                                        LogisticRegression(random_state=333))]),
             n_jobs=3,
             param_grid={'logreg__C': [0.1, 1, 10],
                         'logreg__class_weight': [{'favored': 1,
                                                   'underdog': 10},
                                                  'balanced', None],
                         'logreg__max_iter': [100, 250],
                         'logreg__penalty': ['l2', 'l1', 'elasticnet'],
                         'logreg__solver': ['lbfgs', 'sag', 'saga',
                                            'newton-cholesky']},
             scoring=make_scorer(precision_score, pos_label=underdog))

In [279]:
print(logreg_grid.best_estimator_)
print(logreg_grid.best_score_)

Pipeline(steps=[('scaler', StandardScaler()),
                ('logreg',
                 LogisticRegression(C=0.1, class_weight='balanced',
                                    penalty='l1', random_state=333,
                                    solver='saga'))])
0.41604184150615814
